In [1]:
from sklearn.cluster import KMeans
import numpy as np
import scipy as sp
import sys
sys.path.append('../corv/src')
sys.path.append('../wdtools')

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm
import pandas as pd
import corv

### Query
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c
import data_selector as ds

import wdtools
import corv


using local paths
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


2022-07-28 17:59:54.329458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-28 17:59:54.329480: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Please install galpy if you want to use the gaia module. otherwise, ignore this.


In [2]:
#d1 = pd.read_csv('data/wd_spectra.csv')
#d1 = Table.from_pandas(d1)

catalog = Table.read('data/00_MANUAL_MS.fits')
print(len(catalog))

22318


In [3]:
#spAll = Table.read('~/Downloads/spAll-master.fits')

In [4]:
'''
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join_skycoord
from astropy import table

print(spAll.keys())

print(spAll[['RACAT', 'DECCAT']])

spAll = spAll[spAll['DECCAT'] < 90]
spAll = spAll[spAll['DECCAT'] > -90]

catalog['wd_pos'] = SkyCoord(catalog['wd_ra'], catalog['wd_dec'], unit='deg')
spAll['wd_pos'] = SkyCoord(spAll['RACAT'], spAll['DECCAT'], unit='deg')

join_func = table.join_skycoord(5 * u.arcsecond)
sdss5_catalog = table.join(catalog, spAll, join_funcs={'wd_pos': join_skycoord(5 * u.arcsec)})
'''

"\nfrom astropy.coordinates import SkyCoord\nimport astropy.units as u\nfrom astropy.table import Table, join_skycoord\nfrom astropy import table\n\nprint(spAll.keys())\n\nprint(spAll[['RACAT', 'DECCAT']])\n\nspAll = spAll[spAll['DECCAT'] < 90]\nspAll = spAll[spAll['DECCAT'] > -90]\n\ncatalog['wd_pos'] = SkyCoord(catalog['wd_ra'], catalog['wd_dec'], unit='deg')\nspAll['wd_pos'] = SkyCoord(spAll['RACAT'], spAll['DECCAT'], unit='deg')\n\njoin_func = table.join_skycoord(5 * u.arcsecond)\nsdss5_catalog = table.join(catalog, spAll, join_funcs={'wd_pos': join_skycoord(5 * u.arcsec)})\n"

# GET MS STAR SPECTRA

In [5]:
from astropy.table import Table, vstack, hstack

stardats = []
iters = len(catalog) // 100

for i in tqdm(range(iters)):
    ADQL_CODE1 = "SELECT \
        sdss.original_ext_source_id as bestobjid, gaia_source.source_id\
        FROM gaiaedr3.gaia_source \
        JOIN gaiaedr3.sdssdr13_best_neighbour as sdss\
        ON gaia_source.source_id = sdss.source_id      \
        WHERE gaia_source.source_id IN {}\
    ".format(tuple(catalog['ms_source_id'][100*i:100*i+100]))
    
    job1 = Gaia.launch_job(ADQL_CODE1,dump_to_file=False)
    d1 = job1.get_results()
    
    stardats.append(d1)
d1 = vstack(stardats)


100%|█████████████████████████████████████████████████████████████████████████| 223/223 [03:47<00:00,  1.02s/it]


In [6]:
drops = []
bestobjid1 = []

for i in tqdm (range(len(catalog))):
    notfound = False
    a = np.where(d1['source_id'] == catalog['ms_source_id'][i])
    
    try:
        j = a[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            bestobjid1.append(d1['bestobjid'][j])
        except:
            notfound = True
    if notfound:
        drops.append(i)    
        
catalog.remove_rows(drops)
catalog['ms_bestobjid'] = bestobjid1

100%|███████████████████████████████████████████████████████████████████| 22318/22318 [00:02<00:00, 7760.86it/s]


In [7]:
print(catalog['ms_bestobjid'])

    ms_bestobjid   
-------------------
1237666227511361910
1237666229122957712
1237666210332411647
1237649745022157264
1237658188928779132
1237667138566815852
1237666464256033355
1237667136955416660
1237667138567406889
1237666464793363110
1237666465330299039
                ...
1237651497370255752
1237663530257481761
1237651273496330297
1237651496834171161
1237663788492259646
1237651497369403451
1237663546900283548
1237663787955978272
1237663916801327241
1237663531330765130
1237663916264653725
Length = 6869 rows


In [8]:
stardats2 = []
iters = len(catalog) // 100

for i in tqdm(range(iters)):
    SDSS_QUERY = """select bestObjID, plate, mjd, fiberID, subClass
        from dbo.SpecObjAll
        where bestObjID in {}""".format(tuple(catalog['ms_bestobjid'][100*i:100*i+100]))
    
    dat = SDSS.query_sql(SDSS_QUERY)
    if dat is not None:
        stardats2.append(dat)
spec = vstack(stardats2)

  0%|                                                                                    | 0/68 [00:00<?, ?it/s]/home/arseneausm/anaconda3/lib/python3.8/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|██████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 914.04it/s]


In [9]:
print(spec)


     bestObjID      plate  mjd  fiberID    subClass   
------------------- ----- ----- ------- --------------
1237667112799568095  2345 53757     309             M5
1237667255616405856  2356 53786     338             M6
1237671125375516686  2580 54092     447             M4
1237663278463713420  1488 52965     538             M4
1237663783670513813  1078 52643     291             M2
1237651754532930221   480 51989     609             M8
1237651754532930221  4743 55645     696 M6III (148783)
1237667210511646904  2290 53727     163             M2
1237666339190669360  4228 55484     238 M5III (221615)
1237663784206664016  4229 55501     573 M5III (221615)
1237663784740061261  1131 52902     544             M2
                ...   ...   ...     ...            ...
1237655498667393899   915 52443     200             M9
1237655498667393899  4038 55363     480         Ldwarf
1237655498667393899  4041 55361      20         Ldwarf
1237654391640818073   545 52202     367             M5
1237654653

In [10]:

drops = []
plate = []
mjd = []
fiber = []
types = []

for i in tqdm (range(len(catalog))):
    notfound = False
    a = np.where(spec['bestObjID'] == catalog['ms_bestobjid'][i])
    
    try:
        j = a[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            
            plate.append(spec['plate'][j])
            mjd.append(spec['mjd'][j])
            fiber.append(spec['fiberID'][j])
            types.append(spec['subClass'][j])
            
            
        except:
            notfound = True
    if notfound:
        drops.append(i)    
        
catalog.remove_rows(drops)
catalog['ms_plate'] = plate
catalog['ms_mjd'] = mjd
catalog['ms_fiber'] = fiber
catalog['ms_subClass'] = types


100%|████████████████████████████████████████████████████████████████████| 6869/6869 [00:00<00:00, 39358.04it/s]


In [11]:
print(len(catalog))

50


# GET WD STAR SPECTRA

In [12]:
stardats = []
iters = len(catalog) // 100

for i in tqdm(range(iters)):
    ADQL_CODE1 = "SELECT \
        sdss.original_ext_source_id as bestobjid,\
        gaia_source.source_id\
        FROM gaiaedr3.gaia_source \
        JOIN gaiaedr3.sdssdr13_best_neighbour as sdss\
        ON gaia_source.source_id = sdss.source_id      \
        WHERE gaia_source.source_id IN {}\
    ".format(tuple(catalog['wd_source_id'][100*i:100*i+100]))
    
    job1 = Gaia.launch_job(ADQL_CODE1,dump_to_file=False)
    d2 = job1.get_results()
    stardats.append(d2)
#d1 = vstack(stardats)

print(len(d1))

0it [00:00, ?it/s]

6869


In [13]:
print(len(catalog))

50


In [14]:
drops = []
bestobjid1 = []

for i in tqdm (range(len(catalog))):
    notfound = False
    a = np.where(d1['source_id'] == catalog['wd_source_id'][i])
    
    try:
        j = a[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            bestobjid1.append(d1['bestobjid'][j])
        except:
            notfound = True
    if notfound:
        drops.append(i)    
        
catalog.remove_rows(drops)
catalog['wd_bestobjid'] = bestobjid1

100%|█████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 6782.07it/s]


In [15]:
from astropy.table import Table, vstack, hstack

stardats = []
iters = len(catalog) // 100

for i in tqdm(range(iters)):
    SDSS_QUERY = """select bestObjID, plate, mjd, fiberID, subClass
        from dbo.SpecObjAll
        where bestObjID in {}""".format(tuple(catalog['wd_bestobjid'][100*i:100*i+100]))
    
    stardats.append(SDSS.query_sql(SDSS_QUERY))
spec = vstack(stardats)


0it [00:00, ?it/s]


ValueError: no values provided to stack.

In [ ]:
drops = []
plate = []
mjd = []
fiber = []
types = []

for i in tqdm (range(len(catalog))):
    notfound = False
    a = np.where(spec['bestObjID'] == catalog['wd_bestobjid'][i])
    
    try:
        j = a[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            
            plate.append(spec['plate'][j])
            mjd.append(spec['mjd'][j])
            fiber.append(spec['fiberID'][j])
            types.append(spec['subClass'][j])
            
            
        except:
            notfound = True
    if notfound:
        drops.append(i)    
        
catalog.remove_rows(drops)
catalog['wd_plate'] = plate
catalog['wd_mjd'] = mjd
catalog['wd_fiber'] = fiber
catalog['wd_subClass'] = types

In [ ]:
print(len(catalog))

In [ ]:
spec = []
wl = []
fl = []
ivar = []
drops = []
worked = []

for i in tqdm(range(len(catalog))):
    try:
        spec = SDSS.get_spectra_async(plate=plate[i], fiberID=fiber[i], mjd=mjd[i])[0].get_fits()
        
        #print(spec[4].header)
        
        wl.append(10**spec[1].data['loglam'])
        fl.append(spec[1].data['flux'])
        ivar.append(spec[1].data['ivar'])
        worked.append(i)
    except:
        drops.append(i)
        
print(len(wl))
print(len(fl))
print(len(ivar))
        
catalog.remove_rows(drops)
catalog['wd_wl'] = wl
catalog['wd_fl'] = fl
catalog['wd_ivar'] = ivar

In [ ]:
print(catalog['subClass'])

print(len(catalog))

In [ ]:
def pseudonormalize(fl, win = 50):
    cont_fl = np.repeat(np.nan, len(fl))
    
    for i in range(len(fl)):
        if (i - (win / 2)) < 0:
            fl_win = fl[0:(i + (win // 2))]
        else:
            fl_win = fl[(i - (win // 2)) : (i + (win // 2))]
        
        cont_fl[i] = np.percentile(fl_win, 90)
        
    return fl / cont_fl
'''
def spec_split(bands, wl, fl, window_size = 100):
    spec_split = {}
    
    for i in range(len(bands)):
        spec_split[bands.keys()[i]] = np.array([ [wl[
'''